In [52]:
import pandas as pd
import pprint
import plotly.express as px

In [3]:
df = pd.read_csv('./data/user_song_database.csv', header=None)
df.columns = ['artist', 'track_id', 'track', 'owner', 'popularity']

In [4]:
print(df.shape)
print(df.owner.nunique())

(211801, 4)
4024


In [17]:
owners_count = df.groupby('owner')['track'].count()
fig = px.histogram(owners_count, x='track')
fig.show()

Strangly there seems to be one artist who has a lot of songs?

In [18]:
owners_count.sort_values(ascending=False)

owner
spotify                         42947
christopherkss                    240
nekoni_konbanwa                   234
iloveplaylists                    216
xiytxgvtapjdevr7qezpd90xe         200
                                ...  
3166d2uj5vx2iemlgxnmmdvhj2sq        2
lilyann_kenneally                   2
31idzheq5dm2z3bdnkw3nmtwueea        2
fe907q979eb06frn3dvmme0ux           2
1217143185                          1
Name: track, Length: 4024, dtype: int64

In [19]:
px.histogram(owners_count.drop('spotify'), x='track')

As expected most playlist owners have only 50 songs to them, this is due to how the data was downloaded. Consult `load_data.py`. Some of the playlist owners with more than 50 tracks to them, have had more than 1 playlist of theirs read in.
If `spotify` is included in the model it could distort it hugely, however down the line it would be interesting to see the impact of its inclusion.

In [16]:
# checking how many songs we'd have left if 'spotify' is dropped
owners_count.drop('spotify').sum()

168788

Over 150k songs left in the database! Hence `spotify` will be dropped from now on.

In [25]:
df = df[df['owner'] != 'spotify']

In [29]:
# checking how many duplicate tracks we have
print(df.track_id.shape)
print(df.track_id.nunique())
df.track_id.value_counts()

(168853,)
114915


1Y8KxQUmP1xuaF5mNTYE0x    60
3nhNVCiyHSYbYPFwoKbOA0    50
0QNkVh7nw7KkQSPkjharYW    49
4N28QlYB0XW6H5gaXBHt6T    40
3ilkEyg6OCtd9qCnOJkPzU    34
                          ..
1AXJqw8lCf3chqvgq4U8mA     1
3QyoC6OvQUmpQwQZ18iaTs     1
61WE54H3MD9N5P56Mpr6TN     1
3261Kuksr5Yn8nToKvUW12     1
6ar3x1gVqkABR1x6zQI9yH     1
Name: track_id, Length: 114915, dtype: int64

In [30]:
print(df.track.shape)
print(df.track.nunique())
df.track.value_counts()

(168853,)
89447


Home                             85
Intro                            85
Closer                           67
Winona                           65
riBBon                           60
                                 ..
Groovski                          1
I Feel For You - Star B Remix     1
A Drug From God (SOSA Remix)      1
La Casa                           1
The Hissing of Summer Lawns       1
Name: track, Length: 89447, dtype: int64

In [35]:
df[df['track'] == 'Home']['track_id'].nunique()

56

Clearly there are a few tracks that have the same name but are actually different tracks. As can be seen above, where there are 56 tracks with the title `Home`. So that the model doesn't get confused we could combine the track&artists column into one. Simply using the track_id I think would decrease the information the model would see.

In [46]:
df['artist&track'] = df['artist'] + '/' + df['track']

/tmp/ipykernel_10778/3845966489.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
df.to_csv('./data/database_model_ready.csv', index=False)